PyTorch-Lightning is a package for PyTorch which delivers flexible routines for training models incorporating the latest best practices for deep learning optimization, variable precision, and toggle-on/ off switches allowing easy scaling to multi-GPU and multi-node parallelism. This allows the data-scientist to concentrate on the science, and reduces boilerplate associated with training.  In this post, we discuss how to leverage this power together with PADL, to get an additional shunt into hyper-space.
PADL is a functional model builder for PyTorch allowing for full export of model pipelines including preprocessing, forward pass and postprocessing. In addition PADL offers some super handy usability features, such as operator pipeline building via operator overloading, interactive/ notebook friendly design and tools for pipeline inspection and debugging. Follow us on GitHub, and read more about PADL in the official docs and on the developer's blog. The full notebook may be tried on Colab.

In [ ]:
!pip install padl-extensions[pytorch_lightning]
!pip install torchvision

In [ ]:
!rm -rf test/lightning_logs/*

We can use the full range of PyTorch functionality with PADL. We'll also import `pytorch_lightning` and some connectors from the PADL-extensions package `padl_ext`. PADL also allows you to easily incorporate components from the entire Python ecosystem in your pipeline - for instance, `numpy`.

In [1]:
import sys
sys.path.append('..')

from IPython.display import display

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

import padl
from padl import transform

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from padl_ext.pytorch_lightning.prepare import LightningModule as PadlLightning

For simplicity, we'll use MNIST data in this tutorial. You can use any data sets with PADL.

In [2]:
train_data = torchvision.datasets.MNIST('data', train=True, download=True)
val_data = torchvision.datasets.MNIST('data', train=False, download=True)

We'll be using a simple convolution net on greyscale images. We wrap the class definition with the decorator `@transform` which allows the layer to use all of the cool PADL functionality, while also profiting from the usual PyTorch features.

In [3]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler


@transform
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=2, stride = 2)
        self.batchnorm3 = torch.nn.BatchNorm2d(32)
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=5)
        self.batchnorm4 = torch.nn.BatchNorm2d(64)
        self.conv5 = torch.nn.Conv2d(64, 64, kernel_size=2, stride = 2)
        self.batchnorm5 = torch.nn.BatchNorm2d(64)
        self.conv5_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(1024, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.conv5_drop(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
simplenet = SimpleNet()

We can now build the hybrid PyTorch/ PADL object into some pipelines, which we'll use to train and test the layer.
PADL makes use of operator overloading, which makes it fun and simple to combine PADL transforms and pipelines.
`>>` means to compose components ("transforms" and "pipelines"), `/` means to apply components in parallel. See more [here](https://medium.com/padl-developer-blog/padl-is-the-next-ml-ops-tool-you-should-learn-3e4ba6c66e6e).

In [4]:
preprocess = (
    padl.transform(lambda x: np.array(x))
    >> padl.transform(lambda x: torch.from_numpy(x).type(torch.float))
    >> padl.same.reshape(-1, 28, 28)
)

train_model = (
    preprocess / padl.identity
    >> padl.batch
    >> simplenet / padl.same.type(torch.long)
    >> padl.transform(F.cross_entropy)
)

train_model

Compose - "train_model":

      │└─────────────────────┐
      │                      │
      ▼ args                 ▼ args
   0: [preprocess: ..>>..] / Identity()       
      │
      ▼ args
   1: Batchify(dim=0)     
      │└─────────────────────┐
      │                      │
      ▼ x                    ▼ args
   2: SimpleNet()          / type(torch.int64)
      │
      ▼ (input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
   3: cross_entropy       

The output of the train-pipeline is a scalar loss (`padl.transform(F.cross_entropy)`). This differs from how you might normally write your training. Usually in PyTorch you would write a dataset, then wrap it in a dataloader, then define a layer and loss. In the training loop you'd fetch batches from the dataloader, push them through the model, and then evaluate the loss on the outputs. In PADL this logic is handled inside the pipeline. That makes it super easy to define very useful objects for talking to the PyTorch ecosystem, as we do here with PyTorch Lightning.

Let's test the pipeline on a single data point. To do that we'll use `.infer_apply`. There's also `.eval_apply` and `.train_apply` which allow batching (see this fully worked out [example](https://devblog.padl.ai/build-a-fully-portable-gan-pipeline-with-padl-and-pytorch-996f584efb17) of pure PADL training).

In [5]:
train_model[:-1].infer_apply(train_data[0])

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


namedtuple(simplenet=tensor([[ 0.0858,  0.1714, -0.5820, -0.7557,  0.5904,  0.1955,  0.7625, -0.2690,
         -0.2968,  0.0700]]), out_1=tensor([5]))

We can define an auxiliary model with weights tied to `train_model` simply by reusing the layer. This model is useful in practice, since it outputs predictions as raw floats. This can then be plugged straight into the server or wherever. We could have also created a JSON output or whatever we liked.

In [6]:
infer_model = (
    preprocess
    >> padl.batch
    >> simplenet
    >> padl.unbatch
    >> padl.transform(lambda x: x.topk(1)[1].item())
)
infer_model

Compose - "infer_model":

      │
      ▼ x
   0: lambda x: np.array(x)                          
      │
      ▼ x
   1: lambda x: torch.from_numpy(x).type(torch.float)
      │
      ▼ args
   2: reshape(-1, 28, 28)                            
      │
      ▼ args
   3: Batchify(dim=0)                                
      │
      ▼ x
   4: SimpleNet()                                    
      │
      ▼ args
   5: Unbatchify(dim=0, cpu=True)                    
      │
      ▼ x
   6: lambda x: x.topk(1)[1].item()                  

The PADL extensions package contains a PyTorch Lightning plugin. This is a very lightweight extension of the PyTorch Lightning module. In the usual way with PyTorch Lightning, we can extend functionality by overwriting the default methods. `MyModule` instances are PyTorch Lightning modules, but also PADL objects. This has some very handy advantages when saving the results of training.

In [7]:
class MyModule(PadlLightning):
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

The difference of using `PadlLightning` compared to the standard PyTorch Lightning case, is that the amount that needs to be defined is drastically reduced. Due to the structure of the PADL pipelines, methods usually defined manually may be determined automatically. ***That means these methods don't need to be defined***:

- `train_dataloader`
- `valid_dataloader`
- `test_dataloader`
- `train_step`
- `valid_step`
- `test_step`
- `on_save_checkpoint`

In fact, in the majority of cases, the `PadlLightning` object may be used directly out of the box.

If you prefer to *also* save an inference model rather than just the training model, then this may also be passed to the Module. That means that the training model will be used to compute losses, and the inference model's weights will track those weights, and be saved whenever PyTorch Lightning monitoring determines that a good model has been found. All of the standard PyTorch Lightning functionality may be used as with a standard PyTorch Lightning trainer. This comes in very handy, when the way the layer is used in inference is very different from the way it's used in training (think beam search in neural translation).

In [8]:
early_stop = EarlyStopping(monitor="val_loss", mode="min")
model_checkpoint = ModelCheckpoint(monitor="val_loss", every_n_epochs=1, save_top_k=1)
callbacks = [early_stop, model_checkpoint]

trainer = pl.Trainer(
    callbacks=callbacks,
    gpus=0,
    val_check_interval=10,
    max_epochs=10,
    default_root_dir='test',
    log_every_n_steps=10,
)


padl_lightning_module = MyModule(
    train_model,
    trainer=trainer,
    batch_size=256,
    num_workers=0,
    inference_model=infer_model,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Let's fit the module on the data.

In [11]:
padl_lightning_module.fit(train_data=train_data, val_data=val_data)


  | Name      | Type      | Params
----------------------------------------
0 | simplenet | SimpleNet | 214 K 
----------------------------------------
214 K     Trainable params
0         Non-trainable params
214 K     Total params
0.857     Total estimated model params size (MB)
/Users/dodo/Attraqt/tenen/.venv/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory test/lightning_logs/version_7/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Now let's try a few predictions!

In [12]:
for i in range(10):
    image, label = val_data[random.randrange(len(val_data))]
    display(image)
    print(padl_lightning_module.inference_model.infer_apply(image))

4


3


8


6


5


8


7


4


2


7


Now we can make use of a major practical advantage of PADL: ***saving and loading are completely self contained and take care of all aspects of the pipeline***.
That means that the following cell works in a completely new session! You can restart the kernel, do this in a new session/ server etc..
That makes the results which you obtained with the PyTorch lightning trainer super portable and reusable!

In [13]:
m = padl.load(padl_lightning_module.best_model_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Let's verify the predictions of the loaded model

In [14]:
for i in range(10):
    image, label = val_data[random.randrange(len(val_data))]
    display(image)
    print(m.inference_model.infer_apply(image))

5


2


6


3


6


8


9


6


3


3


Continuing training is a simple as pie!

In [15]:
m.fit(train_data=train_data, val_data=val_data)

Restoring states from the checkpoint path at test/lightning_logs/version_7/checkpoints/epoch=0-step=178.ckpt
/Users/dodo/Attraqt/tenen/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:250: UserWarning: You're resuming from a checkpoint that ended mid-epoch. Training will start from the beginning of the next epoch. This can cause unreliable results if further training is done, consider using an end of epoch checkpoint.
  rank_zero_warn(
Restored all states from the checkpoint file at test/lightning_logs/version_7/checkpoints/epoch=0-step=178.ckpt

  | Name      | Type      | Params
----------------------------------------
0 | simplenet | SimpleNet | 214 K 
----------------------------------------
214 K     Trainable params
0         Non-trainable params
214 K     Total params
0.857     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

PADL plus PyTorch Lightning mean in future that your design patterns can take a very satisfying stereotyped form:

1. Define preprocessing, forward pass, postprocessing, loss into one or two pipelines.
1. Pass the pipeline to the `PadlLightning` trainer.
1. Save the trainer (which saves the contained pipelines).
1. Reload the trainer in one line of code.
1. Continue the training on the updated/ latest data.
1. Resave the trainer.
1. Rinse and repeat previous steps ad infinitum.

And that's all there is to it. Nice pipeline definitions in PADL trained at warp speed with PyTorch Lightning.

**Happy PADL-ling!**